In [ ]:
#!pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 5.3 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619406 sha256=4b5f3f35bfe14e534e356ecaa052381483d8a04c834a9622e19f7807599fab2c
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [1]:
import surprise

In [2]:
print(surprise.__version__)

1.1.1


In [3]:
from surprise import SVD 
## 고유값분해(행렬을 더 낮은 차원으로 분해)해서 다시 원본으로 살리면서 잠재적인 값을 추론하는 방법
from surprise import Dataset, Reader ## SVD를 사용하기 위한 데이터셋 만들어주는 클래스
                            ## 사용자, 아이템, 평점
from surprise import accuracy ## RMSE, MAE, cross-validation(k-fold)
from surprise.model_selection import train_test_split ## 훈련/검증 데이터 분류

In [4]:
## 1. 데이터셋을 만들어주자(사용자, 아이템, 평점),  훈련/검증 데이터 분류
data = Dataset.load_builtin('ml-100k')
data

Dataset ml-100k could not be found. Do you want to download it? [Y/n] 

 y


Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /Users/administrator/.surprise_data/ml-100k


In [5]:
trainset, testset = train_test_split(data, test_size= 0.25, random_state=0)

In [6]:
trainset

In [7]:
testset[:5]

[('120', '282', 4.0),
 ('882', '291', 4.0),
 ('535', '507', 5.0),
 ('697', '244', 5.0),
 ('751', '385', 4.0)]

In [8]:
## 2. SVD() 모델 선정
algo = SVD()

In [9]:
## 3. 훈련용 데이터로 fit()
algo.fit(trainset)

In [10]:
## 4. 검증용 데이터로 예측
predictions = algo.test(testset)[:5]
predictions 

[Prediction(uid='120', iid='282', r_ui=4.0, est=3.645440327343681, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.7360550684390477, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.217392422431921, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.3859664316067946, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.3578952254030887, details={'was_impossible': False})]

In [11]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions]

[('120', '282', 3.645440327343681),
 ('882', '291', 3.7360550684390477),
 ('535', '507', 4.217392422431921),
 ('697', '244', 3.3859664316067946),
 ('751', '385', 3.3578952254030887)]

In [12]:
## 5. 정확도 계산 

In [13]:
## 6. 사용자, 아이템정보 주고 예측 실행
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
pred

Prediction(uid='196', iid='302', r_ui=None, est=4.139033164031143, details={'was_impossible': False})

In [14]:
import pandas as pd

In [15]:
## csv를 SVD하기 위한 DataSet를 주기 위해서는
## 1) df <--- csv, 2) DataSet <--- df

In [19]:
ratings = pd.read_csv('../csv-data/ratings.csv')
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [20]:
reader = Reader(rating_scale=(0.5, 5.0))

In [21]:
## SVD에서 사용할 수 있는 데이터 셋으로 만들어주자.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
data

In [22]:
## 데이터 분류, 객체생성, 훈련시키고, 검증해보고, 검증결과 프린트

In [23]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

In [25]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='../csv-data/ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()

In [26]:
algo = SVD(n_factors=20, n_epochs= 30,  random_state=1)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 1.0388


1.0387802469854106

In [27]:
from surprise.model_selection import cross_validate

In [28]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True) 

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8687  0.8634  0.8680  0.8672  0.8653  0.8665  0.0019  
MAE (testset)     0.6663  0.6635  0.6666  0.6654  0.6637  0.6651  0.0013  
Fit time          2.50    2.63    2.57    2.51    2.50    2.54    0.05    
Test time         0.18    0.17    0.10    0.18    0.10    0.14    0.04    


{'test_rmse': array([0.86869109, 0.86341732, 0.86800138, 0.86717483, 0.86532293]),
 'test_mae': array([0.66626189, 0.66345532, 0.66658928, 0.66535422, 0.6637476 ]),
 'fit_time': (2.498642921447754,
  2.6303188800811768,
  2.5733449459075928,
  2.5075740814208984,
  2.4961209297180176),
 'test_time': (0.18333005905151367,
  0.16707897186279297,
  0.09648990631103516,
  0.1802690029144287,
  0.09549283981323242)}

In [29]:
import time

In [30]:
start = time.time()
start

1652925777.333598

In [31]:
end = time.time()
end

1652925778.2134988

In [32]:
end-start

0.8799009323120117

In [33]:
end2 = time.time()
end2

1652925780.190849

In [34]:
end2-start

2.8572511672973633

In [36]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('../csv-data/movies.csv')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [37]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.50   {'was_impossible': False}


In [38]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


In [39]:
# ## 안본 영화중에서 평점 예측이 높게 나온 3개를 리스트업하는 함수.
# def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n = 5):
#   ## 안본 영화리스트를 하나씩 꺼낸다음 평점을 예측하세요.
#   predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

#   def sortkey_est(one):
#     return one.est

#   predictions.sort(key= sortkey_est, reverse=True)
#   top_predictions = predictions[:top_n]
#   ## print(len(top_predictions))
#   ## print(top_predictions)
#   return top_predictions

In [40]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=5):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(one):
        return one.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    print(list(zip(top_movie_ids, top_movie_titles, top_movie_rating)))
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds


In [41]:
top_predictions = recomm_movie_by_surprise(algo, 9, unseen_movies)
top_predictions

[(1, 'Toy Story (1995)', 3.5012582280677833), (2, 'Jumanji (1995)', 3.5012582280677833), (3, 'Grumpier Old Men (1995)', 3.5012582280677833), (4, 'Waiting to Exhale (1995)', 3.5012582280677833), (5, 'Father of the Bride Part II (1995)', 3.5012582280677833)]


[(1, 'Toy Story (1995)', 3.5012582280677833),
 (2, 'Jumanji (1995)', 3.5012582280677833),
 (3, 'Grumpier Old Men (1995)', 3.5012582280677833),
 (4, 'Waiting to Exhale (1995)', 3.5012582280677833),
 (5, 'Father of the Bride Part II (1995)', 3.5012582280677833)]